## Data Transformation

In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.svm import LinearSVC
from pandas.core.frame import DataFrame
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
def disease_dict(column):
    count=0
    disease_set = set()
    disease_dict = {}
    for line in column:
        if line==None:
            continue
        else:
            disease_set.add(line)
    for item in disease_set:
        count += 1
        disease_dict[item]=count
    return disease_dict


In [4]:
# Import dataset and count the null value
data = pd.read_csv('final_result.csv')
data = data.drop(columns=['DAYSLAG_DIS_PREVDIS','SEQ_NO','SEQ_NO_1'])
data['PBSDISEASEGROUP'].isnull().value_counts()

False    2495551
Name: PBSDISEASEGROUP, dtype: int64

In [4]:
##If does not already got the disease dictonary run this code
# disease_list = disease_dict(data['PBSDISEASEGROUP'])
# with open('final_disease_dic.json','w') as f:
#     json.dump(disease_list, f)

In [5]:
#Otherwise, just open the dictonary
with open('final_disease_dic.json','r', encoding='utf-8') as f:
    disease_list = json.load(f)

In [6]:
#clean dataset
#replace disease with dictonary
data.replace(disease_list,inplace = True)

# sort
data_sorted = data.sort_values(
    ['MASTERPATIENTID', 'DISPENSECALENDARDATE'], ascending=[1, 1])

# remove duplicate
data_sorted.drop_duplicates(
    subset=['MASTERPATIENTID',
        'PBSDISEASEGROUP'],
    keep='first',
    inplace=True)

new_data = data_sorted.drop(columns=['DISPENSECALENDARDATE'])
#group by patient id and make list of disease
data_new = new_data.groupby(['MASTERPATIENTID'])['PBSDISEASEGROUP'].apply(list)
#number of distinct patient
data_new.shape

(258252,)

In [7]:
#transform data set
transform = data_sorted.drop(columns=['DISPENSECALENDARDATE'])
transform['count1'] = 1
transform.set_index('MASTERPATIENTID',inplace = True)
transform = transform.set_index('PBSDISEASEGROUP',append = True)
transform = pd.Series(transform.values.reshape(len(transform)),index = transform.index)
transform = transform.unstack()
transform = transform.fillna(0)

##deleting the last value y
i=0
for id in data_new.keys():
    transform[data_new[id][-1]][i] = 0.00000
    i+=1

without_zero = transform.ix[~(transform==0).all(axis=1),:]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [8]:
# y dataframe
y_list = []
for id in data_new.keys():
    if len(data_new[id]) == 1:
        continue
    y_list.append(data_new[id][-1])
c={"y" : y_list}
y_result = pd.DataFrame(c)

# adding y value to the dataset
new_transform = without_zero
new_transform = new_transform.reset_index(drop=True)
new_transform = pd.concat([new_transform,y_result], axis=1)
x = new_transform.iloc[:, :-1]
y = new_transform.iloc[:, -1]

In [9]:
#split into train data and test data 20%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Logistic Regression

In [11]:
clf_lr = LogisticRegression(solver='lbfgs',multi_class='multinomial')
clf_lr = clf_lr.fit(x_train, y_train)
y_predict_clf_lr_prob = clf_lr.predict_proba(x_test)
y_predict_clf_lr = clf_lr.predict(x_test)
accuracy_score(y_test, y_predict_clf_lr)

0.3066349151032894

## Linear SVC

In [12]:
clf_linearSVC = LinearSVC(multi_class='ovr')
clf_linearSVC = clf_linearSVC.fit(x_train, y_train)
y_predict_linearSVC = clf_linearSVC.predict(x_test)
accuracy_score(y_test, y_predict_linearSVC)

0.31081682832859